In [1]:
import sys
import numpy as np
import xarray as xr

sys.path.append(".")
from automatic_setup import wrapper_zoo_model
from seapopym.configuration.no_transport.parameter import ForcingParameters
from seapopym.configuration.parameters.parameter_forcing import ForcingUnit
from seapopym.logging.custom_logger import set_error
from seapopym.standard.units import StandardUnitsLabels
from deap import base
from deap import creator
from deap import tools
from deap import algorithms
import numpy as np
import random

random.seed(64)

In [2]:
set_error()
xr.set_options(
    keep_attrs=True,
    display_expand_attrs=False,
    display_expand_data=False,
    display_expand_coords=False,
    display_expand_data_vars=False,
)

WET_TO_CARBON = 1 / 11.9

## Forcing


Forcing data from seapodym model


In [3]:
hot_data = xr.open_dataset(
    "../1_data_processing/1_1_Forcing/hot_cmems.zarr", engine="zarr"
)
hot_data["T"].attrs["units"] = StandardUnitsLabels.temperature.units
hot_data.load()

<xarray.Dataset> Size: 829kB
Dimensions:              (time: 9011, depth: 3, latitude: 1, longitude: 1)
Coordinates: (4)
Data variables: (13)
Attributes: (10)

Observation data from Hot station


In [4]:
hot_obs = xr.open_dataset(
    "../1_data_processing/1_1_Forcing/hot_obs.zarr", engine="zarr"
)
hot_obs.load()

<xarray.Dataset> Size: 2MB
Dimensions:        (time: 659, latitude: 1, longitude: 1, depth: 8, frac: 5)
Coordinates: (5)
Data variables: (24)

Generate the observed zooplankton data to be used further in the cost function


In [5]:
zoo_obs = (
    (hot_obs["dwt_migrant"] + hot_obs["dwt_resident"])
    .sum("frac")
    .sum("depth")
    .cf.isel(Y=0, X=0)
    .pint.quantify()
    .pint.to("kg/m^2")
)
zoo_obs = zoo_obs.where(zoo_obs > 0, drop=True) * WET_TO_CARBON
zoo_obs

<xarray.DataArray (time: 458)> Size: 4kB
[kg/m²] 6.136e-05 9.605e-06 4.229e-05 ... 0.0002705 8.388e-05 0.0001412
Coordinates: (3)
Attributes: (36)

## Setup


The forcing data


In [6]:
hot_data_parameter = ForcingParameters(
    temperature=ForcingUnit(forcing=hot_data["T"], resolution=0.08333),
    primary_production=ForcingUnit(forcing=hot_data["npp"], resolution=0.08333),
)

The initial conditions


In [7]:
setup_model = wrapper_zoo_model(
    tr_max=10.38,
    tr_rate=-0.11,
    inv_lambda_max=150,
    inv_lambda_rate=0.15,
    forcing_parameter=hot_data_parameter,
)
initial_conditions = setup_model.export_initial_conditions()

Update the forcing data with initial conditions


In [8]:
hot_data_parameter = ForcingParameters(
    temperature=ForcingUnit(forcing=hot_data["T"], resolution=0.08333),
    primary_production=ForcingUnit(forcing=hot_data["npp"], resolution=0.08333),
    initial_condition_biomass=ForcingUnit(
        forcing=initial_conditions["initial_condition_biomass"], resolution=0.08333
    ),
    initial_condition_production=ForcingUnit(
        forcing=initial_conditions["initial_condition_production"], resolution=0.08333
    ),
)

## Cost function


In [9]:
def cost_function(args: tuple[float, float, float, float]):
    """Use the Mean Absolute Error (MAE) method."""

    tr_max, tr_rate, inv_lambda_max, inv_lambda_rate = args

    setup_model = wrapper_zoo_model(
        tr_max=tr_max,
        tr_rate=tr_rate,
        inv_lambda_max=inv_lambda_max,
        inv_lambda_rate=inv_lambda_rate,
        forcing_parameter=hot_data_parameter,
    )
    biomass_pred = (
        setup_model.export_biomass()
        .cf.isel(functional_group=0, X=0, Y=0)
        .pint.quantify()
    )
    cost = float(np.abs(zoo_obs - biomass_pred).mean())
    return (cost,)

## DEAP - Genetic algorithm


Create the population of the genetic algorithm.


In [10]:
toolbox = base.Toolbox()

creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox.register("tr_max", random.uniform, 1, 100)
toolbox.register("tr_rate", random.uniform, -1, -0.01)
toolbox.register("inv_lambda_max", random.uniform, 1, 1000)
toolbox.register("inv_lambda_rate", random.uniform, 0.01, 1)

toolbox.register(
    "individual",
    tools.initCycle,
    creator.Individual,
    (toolbox.tr_max, toolbox.tr_rate, toolbox.inv_lambda_max, toolbox.inv_lambda_rate),
    n=1,
)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

And now the cost function and behavior of the genetic algorithm.


In [11]:
toolbox.register("evaluate", cost_function)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

The main function to run the genetic algorithm.


In [12]:
def main():
    pop = toolbox.population(n=10)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    pop, log = algorithms.eaSimple(
        pop,
        toolbox,
        cxpb=0.5,
        mutpb=0.2,
        ngen=40,
        stats=stats,
        halloffame=hof,
        verbose=True,
    )
    return pop, log, hof

In [13]:
pop, log, hof = main()

gen	nevals	avg        	std        	min        	max       
0  	10    	0.000255389	0.000525398	5.77245e-05	0.00182971
1  	6     	0.00043366 	0.00069843 	7.45895e-05	0.00182971


ValueError: 'temperature_recruitment_max' must be > 0: 0.0